In [2]:
# Datasets auxiliares que faltan: 
#- Equipo nombre completo + 3 letras
#- Posición equipo NBA en cada año

# Coses a fer per tenir el cleaning:
# Alguns anys de draft el número de pick el tenen incomplet (aplicar ordre)
# Cal fer correspondencia dels equips antics desapareguts i nous (Draft i basic -> nova columna)

#Coses per a fer al feature engineering
#Afegir columna de has played play-off
#scrapejar número de títols de la NBA

## 46-49 es ABA y no NBA -> borramos registros de antes??? (cleaning)

In [1]:
#pip install progressbar
#pip install XXXXXXXXXXX [other packages needed]

import numpy as np
import pandas as pd
from bs4 import BeautifulSoup, SoupStrainer
import sys
import string
import requests
import datetime
import progressbar
import time
import re

In [2]:
def player_basic_info():
    players = []
    base_url = 'http://www.basketball-reference.com/players/' # this is the page where we are going to find every player in the NBA and ABA history
    start_timer = time.perf_counter()
    for letter in string.ascii_lowercase:
        print('Scrapping basic info of players with letter ' + letter + '...')
        page_request = requests.get(base_url + letter) #we will navigate by letter
        soup = BeautifulSoup(page_request.text,"lxml")
        table = soup.find('table')
        if table:
            table_body = table.find('tbody')
            for row in table_body.findAll('tr'):
                player_url = row.find('a')
                player_names = player_url.text
                player_pages = player_url['href']
                cells = row.findAll('td') # all data for all players uniform across database
                #the information is displayed in this order:
                active_from = int(cells[0].text)
                active_to = int(cells[1].text)
                position = cells[2].text
                height = cells[3].text
                weight = cells[4].text
                birth_date = cells[5].text
                college = cells[6].text    
                player_entry = {'player_url': player_pages,
                                'player_name': player_names,
                                'active_from': active_from,
                                'active_to': active_to,
                                'position': position,
                                'college': college,# we will have to treat this info (+ one college, consistency in names...)
                                'height': height,
                                'weight': weight,
                                'birth_date': birth_date} # we will have to treat this info (date format)
                players.append(player_entry)
    end_timer = time.perf_counter()
    print(f"Scrapped the basic info of the " + str(len(players)) + f" NBA and ABA players in {end_timer - start_timer:0.4f} seconds")
    return pd.DataFrame(players)

#a[a['name'].str.contains('Rubio')] Proves that the dataset contains the info expected

In [3]:
def get_draft_dataframe():
    start_timer = time.perf_counter()
    page_request = requests.get('https://www.basketball-reference.com/draft/')
    soup = BeautifulSoup(page_request.text,"lxml")
    draft_init_table = soup.find_all("th",{"data-stat" : "draft"})[1:]
    draft_urls = []
    for row in draft_init_table:
        draft_urls.append(row.a['href'])
    
    data = []
    df = pd.DataFrame(data, columns=['draft_year','draft_round', 'draft_pick_number', 'draft_team','player_name','player_url','college'])
    for url in draft_urls:
        df = df.append(get_draft_info_by_year(url))
        
    end_timer = time.perf_counter()
    print(f"Scrapped the Draft info of the " + str(len(df)) + f" drafted NBA players in {end_timer - start_timer:0.4f} seconds")    
    return df
    

In [4]:
def get_draft_info_by_year(url): #ojo pq hi ha files on l'equip va decidir no fer el pick (exemple 2002) 
    page_request = requests.get('https://www.basketball-reference.com/' + url)
    soup = BeautifulSoup(page_request.text,"lxml")
    
    year = url[-9:-5]
    print('Scrapping ' + year + ' Draft info...')
    round = 1
    
    Round = []
    Pick_number = []
    Team = []
    Player_name = []
    Player_url = []
    College = []
    
    table = soup.find("table")
    table_body = table.find('tbody')
    
    for row in table_body.findAll('tr'):
        try:
            # primer comprovem que no hi ha cap error al scrapejar la info, després afegim
            cells = row.findAll('td')
            pick_number = cells[0].text
            team = cells[1].text
            player_name = cells[2].text
            player_url = cells[2].a['href']
            college = cells[3].text
            
            Pick_number.append(pick_number)
            Team.append(team)
            Player_name.append(player_name)
            Player_url.append(player_url)
            College.append(college)
            Round.append(round)
        except:
            try:
                if row.get('class')[0] == "over_header":
                    round = round + 1
            except:
                #print(len(Team))
                pass
    Year = [year] * len(Round)
    #print(len(Year))
    #print(len(Team))
    
    return pd.DataFrame({'draft_year':Year,'draft_round':Round, 'draft_pick_number':Pick_number, 'draft_team':Team,'player_name':Player_name,'player_url':Player_url,'college':College})
    

### Funciones para scrapear estadísticas en NBA

Unas consideraciones previas:
- 1o hay que escoger que tablas vamos a scrapear-
- Hay que tener en cuenta que hay jugadores que van a tener muchas menos estadísticas que los actuales (a partir de finales de los 90). Por varias razones: antiguedad, aun NO EXISTIA el tiro de 3P, rebotes no contados hasta el 50, triples dobles popularizados en 1990, etc... Ejemplos: '/players/a/ablefo01.html', '/players/a/abdulza01.html' 
- Baixa't la funció de empty NaN's de PwC
- Considerar el borrar aquellos registros que no tengan suficientes datos
- Ver como tratamos los '' y los None del dataframe
- Scrapearia un botón de playoff para ver que análisis hago del jugador en concreto (me ahorro tiempo)

In [5]:
def player_info_per_game(player_url,table_id,extra_name): #table_id: 'per_game','playoffs_per_game'...
    
    #We will get the info at the Career footpart
    games = None
    games_started = None
    minutes_played_pg = None
    field_goals_pg = None
    field_goals_attempts_pg = None
    field_goals_percent_pg = None
    _3pts_goals_pg = None
    _3pts_goals_attempts_pg = None
    _3pts_goals_percent_pg = None
    _2pts_goals_pg = None
    _2pts_goals_attempts_pg = None
    _2pts_goals_percent_pg = None
    effective_field_goals_percent_pg = None
    FT_goals_pg = None
    FT_goals_attempts_pg = None
    FT_goals_percent_pg = None
    off_rebounds_pg = None
    def_rebounds_pg = None
    total_rebounds_pg = None
    assists_pg = None
    steals_pg = None
    blocks_pg = None
    turnovers_pg = None
    personal_foults_pg = None
    points_pg = None
    
    page_request = requests.get('https://www.basketball-reference.com' + str(player_url))
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find("table",{'id':table_id})
    
    if table:
            table_foot = table.find('tfoot')
            career_row = table_foot.find('tr')
            cells  = career_row.findAll('td')
            playerData = str(cells) #the indexes are not uniform across the database
            # It searches using regular expressions!!!!!
            games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1)
            games_started = re.search(r'data-stat="gs">(.*?)</td>', playerData).group(1)
            minutes_played_pg = re.search(r'data-stat="mp_per_g">(.*?)</td>', playerData).group(1)
            field_goals_pg = re.search(r'data-stat="fg_per_g">(.*?)</td>', playerData).group(1)
            field_goals_attempts_pg = re.search(r'data-stat="fga_per_g">(.*?)</td>', playerData).group(1)
            field_goals_percent_pg = re.search(r'data-stat="fg_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData) != None:
                _3pts_goals_pg = re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData).group(1)
                _3pts_goals_attempts_pg = re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData).group(1)
                _3pts_goals_percent_pg = re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="fg2_per_g">(.*?)</td>', playerData) != None:
                _2pts_goals_pg = re.search(r'data-stat="fg2_per_g">(.*?)</td>', playerData).group(1)
                _2pts_goals_attempts_pg = re.search(r'data-stat="fg2a_per_g">(.*?)</td>', playerData).group(1)
                _2pts_goals_percent_pg = re.search(r'data-stat="fg2_pct">(.*?)</td>', playerData).group(1)
                effective_field_goals_percent_pg = re.search(r'data-stat="efg_pct">(.*?)</td>', playerData).group(1)
            FT_goals_pg = re.search(r'data-stat="ft_per_g">(.*?)</td>', playerData).group(1)
            FT_goals_attempts_pg = re.search(r'data-stat="fta_per_g">(.*?)</td>', playerData).group(1)
            FT_goals_percent_pg = re.search(r'data-stat="ft_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="orb_per_g">(.*?)</td>', playerData) != None:
                off_rebounds_pg = re.search(r'data-stat="orb_per_g">(.*?)</td>', playerData).group(1)
                def_rebounds_pg = re.search(r'data-stat="drb_per_g">(.*?)</td>', playerData).group(1)
            total_rebounds_pg = re.search(r'data-stat="trb_per_g">(.*?)</td>', playerData).group(1)
            assists_pg = re.search(r'data-stat="ast_per_g">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="stl_per_g">(.*?)</td>', playerData) != None:
                steals_pg = re.search(r'data-stat="stl_per_g">(.*?)</td>', playerData).group(1)
                blocks_pg = re.search(r'data-stat="blk_per_g">(.*?)</td>', playerData).group(1)
                turnovers_pg = re.search(r'data-stat="tov_per_g">(.*?)</td>', playerData).group(1)
            personal_foults_pg = re.search(r'data-stat="pf_per_g">(.*?)</td>', playerData).group(1)
            points_pg = re.search(r'data-stat="pts_per_g">(.*?)</td>', playerData).group(1)
        
    player_info = {'player_url' : player_url,
                   'games' + extra_name: games,
                   'games_started' + extra_name: games_started,
                   'minutes_played_pg' + extra_name: minutes_played_pg,
                   'field_goals_pg' + extra_name: field_goals_pg,
                   'field_goals_attempts_pg' + extra_name: field_goals_attempts_pg,
                   'field_goals_percent_pg' + extra_name: field_goals_percent_pg,
                   '_3pts_goals_pg' + extra_name: _3pts_goals_pg,
                   '_3pts_goals_attempts_pg' + extra_name: _3pts_goals_attempts_pg,
                   '_3pts_goals_percent_pg' + extra_name: _3pts_goals_percent_pg,
                   '_2pts_goals_pg' + extra_name: _2pts_goals_pg,
                   '_2pts_goals_attempts_pg' + extra_name: _2pts_goals_attempts_pg,
                   '_2pts_goals_percent_pg' + extra_name: _2pts_goals_percent_pg,
                   'effective_field_goals_percent_pg' + extra_name: effective_field_goals_percent_pg,
                   'FT_goals_pg' + extra_name: FT_goals_pg,
                   'FT_goals_attempts_pg' + extra_name: FT_goals_attempts_pg,
                   'FT_goals_percent_pg' + extra_name: FT_goals_percent_pg,
                   'off_rebounds_pg' + extra_name: off_rebounds_pg,
                   'def_rebounds_pg' + extra_name: def_rebounds_pg,
                   'total_rebounds_pg' + extra_name: total_rebounds_pg,
                   'assists_pg' + extra_name: assists_pg,
                   'steals_pg' + extra_name: steals_pg,
                   'blocks_pg' + extra_name: blocks_pg,
                   'turnovers_pg' + extra_name: turnovers_pg,
                   'personal_foults_pg' + extra_name: personal_foults_pg,
                   'points_pg' + extra_name: points_pg}
                
    return player_info

In [6]:
def player_info_totals(player_url,table_id,extra_name): #table_id: 'totals','playoffs_totals '...
    
    #We will get the info at the Career footpart
    games = None
    games_started = None
    minutes_played_total = None
    field_goals_total = None
    field_goals_attempts_total = None
    field_goals_percent_total = None
    _3pts_goals_total = None
    _3pts_goals_attempts_total = None
    _3pts_goals_percent_total = None
    _2pts_goals_total = None
    _2pts_goals_attempts_total = None
    _2pts_goals_percent_total = None
    effective_field_goals_percent_total = None
    FT_goals_total = None
    FT_goals_attempts_total = None
    FT_goals_percent_total = None
    off_rebounds_total = None
    def_rebounds_total = None
    total_rebounds_total = None
    assists_total = None
    steals_total = None
    blocks_total = None
    turnovers_total = None
    personal_foults_total = None
    points_total = None
    triple_doubles_total = None 
    
    
    page_request = requests.get('https://www.basketball-reference.com' + str(player_url))
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find("table",{'id':table_id})
    
    if table:
            table_foot = table.find('tfoot')
            career_row = table_foot.find('tr')
            cells  = career_row.findAll('td')
            playerData = str(cells) #the indexes are not uniform across the database
            # It searches using regular expressions!!!!!
            games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1)
            games_started = re.search(r'data-stat="gs">(.*?)</td>', playerData).group(1)
            minutes_played_total = re.search(r'data-stat="mp">(.*?)</td>', playerData).group(1)
            field_goals_total = re.search(r'data-stat="fg">(.*?)</td>', playerData).group(1)
            field_goals_attempts_total = re.search(r'data-stat="fga">(.*?)</td>', playerData).group(1)
            field_goals_percent_total = re.search(r'data-stat="fg_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="fg3">(.*?)</td>', playerData) != None:
                _3pts_goals_total = re.search(r'data-stat="fg3">(.*?)</td>', playerData).group(1)
                _3pts_goals_attempts_total = re.search(r'data-stat="fg3a">(.*?)</td>', playerData).group(1)
                _3pts_goals_percent_total = re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="fg2">(.*?)</td>', playerData) != None:
                _2pts_goals_total = re.search(r'data-stat="fg2">(.*?)</td>', playerData).group(1)
                _2pts_goals_attempts_total = re.search(r'data-stat="fg2a">(.*?)</td>', playerData).group(1)
                _2pts_goals_percent_total = re.search(r'data-stat="fg2_pct">(.*?)</td>', playerData).group(1)
                effective_field_goals_percent_total = re.search(r'data-stat="efg_pct">(.*?)</td>', playerData).group(1)
            FT_goals_total = re.search(r'data-stat="ft">(.*?)</td>', playerData).group(1)
            FT_goals_attempts_total = re.search(r'data-stat="fta">(.*?)</td>', playerData).group(1)
            FT_goals_percent_total = re.search(r'data-stat="ft_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="orb">(.*?)</td>', playerData) != None:
                off_rebounds_total = re.search(r'data-stat="orb">(.*?)</td>', playerData).group(1)
                def_rebounds_total = re.search(r'data-stat="drb">(.*?)</td>', playerData).group(1)
            total_rebounds_total = re.search(r'data-stat="trb">(.*?)</td>', playerData).group(1)
            assists_total = re.search(r'data-stat="ast">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="stl">(.*?)</td>', playerData) != None:
                steals_total = re.search(r'data-stat="stl">(.*?)</td>', playerData).group(1)
                blocks_total = re.search(r'data-stat="blk">(.*?)</td>', playerData).group(1)
                turnovers_total = re.search(r'data-stat="tov">(.*?)</td>', playerData).group(1)
            personal_foults_total = re.search(r'data-stat="pf">(.*?)</td>', playerData).group(1)
            points_total = re.search(r'data-stat="pts">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="trp_dbl">(.*?)</td>', playerData) != None:
                triple_doubles_total = re.search(r'data-stat="trp_dbl">(.*?)</td>', playerData).group(1)
        
    player_info = {'player_url' : player_url,
                   'games' + extra_name: games,
                   'games_started' + extra_name: games_started,
                   'minutes_played_total' + extra_name: minutes_played_total,
                   'field_goals_total' + extra_name: field_goals_total,
                   'field_goals_attempts_total' + extra_name: field_goals_attempts_total,
                   'field_goals_percent_total' + extra_name: field_goals_percent_total,
                   '_3pts_goals_total' + extra_name: _3pts_goals_total,
                   '_3pts_goals_attempts_total' + extra_name: _3pts_goals_attempts_total,
                   '_3pts_goals_percent_total' + extra_name: _3pts_goals_percent_total,
                   '_2pts_goals_total' + extra_name: _2pts_goals_total,
                   '_2pts_goals_attempts_total' + extra_name: _2pts_goals_attempts_total,
                   '_2pts_goals_percent_total' + extra_name: _2pts_goals_percent_total,
                   'effective_field_goals_percent_total' + extra_name: effective_field_goals_percent_total,
                   'FT_goals_total' + extra_name: FT_goals_total,
                   'FT_goals_attempts_total' + extra_name: FT_goals_attempts_total,
                   'FT_goals_percent_total' + extra_name: FT_goals_percent_total,
                   'off_rebounds_total' + extra_name: off_rebounds_total,
                   'def_rebounds_total' + extra_name: def_rebounds_total,
                   'total_rebounds_total' + extra_name: total_rebounds_total,
                   'assists_total' + extra_name: assists_total,
                   'steals_total' + extra_name: steals_total,
                   'blocks_total' + extra_name: blocks_total,
                   'turnovers_total' + extra_name: turnovers_total,
                   'personal_foults_total' + extra_name: personal_foults_total,
                   'points_total' + extra_name: points_total,
                   'triple_doubles_total' + extra_name: triple_doubles_total}
                
    return player_info

In [7]:
def player_info_advanced(player_url,table_id,extra_name): #table_id: 'advanced','playoffs_advanced'...
    
    #We will get the info at the Career footpart
    player_eff_rating = None
    true_shooting_eff = None
    _3pts_attempts_rate = None #
    FT_attempts_rate = None
    off_rebounds_percent = None
    def_rebounds_percent = None
    total_rebounds_percent = None
    assists_percent = None
    steals_percent = None #
    blocks_percent = None #
    turnover_percent = None #
    usage_percent = None #
    off_win_share = None
    def_win_share = None
    win_share = None
    win_share_48min = None
    off_box_pm = None #
    def_box_pm = None #
    box_pm = None #
    value_over_replacement = None #
    
    page_request = requests.get('https://www.basketball-reference.com' + str(player_url))
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find("table",{'id':table_id})
    
    if table:
            table_foot = table.find('tfoot')
            career_row = table_foot.find('tr')
            cells  = career_row.findAll('td')
            playerData = str(cells) #the indexes are not uniform across the database
            # It searches using regular expressions!!!!!
            player_eff_rating = re.search(r'data-stat="per">(.*?)</td>', playerData).group(1)
            true_shooting_eff = re.search(r'data-stat="ts_pct">(.*?)</td>', playerData).group(1)
            FT_attempts_rate = re.search(r'data-stat="fta_per_fga_pct">(.*?)</td>', playerData).group(1)
            off_rebounds_percent = re.search(r'data-stat="orb_pct">(.*?)</td>', playerData).group(1)
            def_rebounds_percent = re.search(r'data-stat="drb_pct">(.*?)</td>', playerData).group(1)
            total_rebounds_percent = re.search(r'data-stat="trb_pct">(.*?)</td>', playerData).group(1)
            assists_percent = re.search(r'data-stat="ast_pct">(.*?)</td>', playerData).group(1)
            off_win_share = re.search(r'data-stat="ows">(.*?)</td>', playerData).group(1)
            def_win_share = re.search(r'data-stat="dws">(.*?)</td>', playerData).group(1)
            win_share = re.search(r'data-stat="ws">(.*?)</td>', playerData).group(1)
            win_share_48min = re.search(r'data-stat="ws_per_48">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="fg3a_per_fga_pct">(.*?)</td>', playerData) != None:
                _3pts_attempts_rate = re.search(r'data-stat="fg3a_per_fga_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="stl_pct">(.*?)</td>', playerData) != None:
                steals_percent = re.search(r'data-stat="stl_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="blk_pct">(.*?)</td>', playerData) != None:
                blocks_percent = re.search(r'data-stat="blk_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="tov_pct">(.*?)</td>', playerData) != None:
                turnover_percent = re.search(r'data-stat="tov_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="usg_pct">(.*?)</td>', playerData) != None:
                usage_percent = re.search(r'data-stat="usg_pct">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="obpm">(.*?)</td>', playerData) != None:
                off_box_pm = re.search(r'data-stat="obpm">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="dbpm">(.*?)</td>', playerData) != None:
                def_box_pm = re.search(r'data-stat="dbpm">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="bpm">(.*?)</td>', playerData) != None:
                box_pm = re.search(r'data-stat="bpm">(.*?)</td>', playerData).group(1)
            if re.search(r'data-stat="vorp">(.*?)</td>', playerData) != None:
                value_over_replacement = re.search(r'data-stat="vorp">(.*?)</td>', playerData).group(1)
        
    player_info = {'player_url' : player_url,
                   'player_eff_rating' + extra_name: player_eff_rating,
                   'true_shooting_eff' + extra_name: true_shooting_eff,
                   '_3pts_attempts_rate' + extra_name: _3pts_attempts_rate,
                   'FT_attempts_rate' + extra_name: FT_attempts_rate,
                   'off_rebounds_percent' + extra_name: off_rebounds_percent,
                   'def_rebounds_percent' + extra_name: def_rebounds_percent,
                   'total_rebounds_percent' + extra_name: total_rebounds_percent,
                   'assists_percent' + extra_name: assists_percent,
                   'steals_percent' + extra_name: steals_percent,
                   'blocks_percent' + extra_name: blocks_percent,
                   'turnover_percent' + extra_name: turnover_percent,
                   'usage_percent' + extra_name: usage_percent,
                   'off_win_share' + extra_name: off_win_share,
                   'def_win_share' + extra_name: def_win_share,
                   'win_share' + extra_name: win_share,
                   'win_share_48min' + extra_name: win_share_48min,
                   'off_box_pm' + extra_name: off_box_pm,
                   'def_box_pm' + extra_name: def_box_pm,
                   'value_over_replacement' + extra_name: box_pm}
    
    return player_info

In [8]:
def player_info_extras(player_url): #ver si queremos scrapear alguna otra información
    
    Hall_Of_Fame = 'No'
    n_NBA_Champ = 0
    n_All_Star = 0
    
    page_request = requests.get('https://www.basketball-reference.com' + str(player_url))
    soup = BeautifulSoup(page_request.text,"lxml")
    extras = soup.find('ul',{'id':'bling'})
    
    if extras:
        if extras.find('li',string = re.compile('Hall of Fame')) != None:  
            Hall_Of_Fame = 'Yes'
        if extras.find('li',string = re.compile('NBA Champ')) != None:
            aux = extras.find('li',string = re.compile('NBA Champ')).text
            n_NBA_Champ = aux[:aux.find('x')]
        if extras.find('li',string = re.compile('All Star')) != None:  
            aux = extras.find('li',string = re.compile('All Star')).text
            n_All_Star = aux[:aux.find('x')]
        
    player_info = {'player_url' : player_url,
                   'Hall_Of_Fame' : Hall_Of_Fame,
                   'n_NBA_Champ' : n_NBA_Champ,
                   'n_All_Star' : n_All_Star}
    
    return player_info

In [ ]:
#FALTA: scrapejar stats college.

### Pruebas de las funciones

In [9]:
original = player_basic_info()

Scrapping basic info of players with letter a...
Scrapping basic info of players with letter b...
Scrapping basic info of players with letter c...
Scrapping basic info of players with letter d...
Scrapping basic info of players with letter e...
Scrapping basic info of players with letter f...
Scrapping basic info of players with letter g...
Scrapping basic info of players with letter h...
Scrapping basic info of players with letter i...
Scrapping basic info of players with letter j...
Scrapping basic info of players with letter k...
Scrapping basic info of players with letter l...
Scrapping basic info of players with letter m...
Scrapping basic info of players with letter n...
Scrapping basic info of players with letter o...
Scrapping basic info of players with letter p...
Scrapping basic info of players with letter q...
Scrapping basic info of players with letter r...
Scrapping basic info of players with letter s...
Scrapping basic info of players with letter t...
Scrapping basic info

In [10]:
original

,player_url,player_name,active_from,active_to,position,college,height,weight,birth_date
0,/players/a/abdelal01.html,Alaa Abdelnaby,1991,1995,F-C,Duke,6-10,240,"June 24, 1968"
1,/players/a/abdulza01.html,Zaid Abdul-Aziz,1969,1978,C-F,Iowa State,6-9,235,"April 7, 1946"
2,/players/a/abdulka01.html,Kareem Abdul-Jabbar,1970,1989,C,UCLA,7-2,225,"April 16, 1947"
3,/players/a/abdulma02.html,Mahmoud Abdul-Rauf,1991,2001,G,LSU,6-1,162,"March 9, 1969"
4,/players/a/abdulta01.html,Tariq Abdul-Wahad,1998,2003,F,"Michigan, San Jose State",6-6,223,"November 3, 1974"
...,...,...,...,...,...,...,...,...,...
4973,/players/z/zizican01.html,Ante Žižić,2018,2020,F-C,,6-10,266,"January 4, 1997"
4974,/players/z/zoetji01.html,Jim Zoet,1983,1983,C,Kent State University,7-1,240,"December 20, 1953"
4975,/players/z/zopfbi01.html,Bill Zopf,1971,1971,G,Duquesne,6-1,170,"June 7, 1948"
4976,/players/z/zubaciv01.html,Ivica Zubac,2017,2022,C,,7-0,240,"March 18, 1997"


In [11]:
#Comprovamos que el campo "player_url" es realmente un identificador del jugador
idd = original.drop_duplicates(subset=['player_url'])
print(len(idd))
print(len(original))

4978
4978


In [12]:
draft = get_draft_dataframe()

Scrapping 2021 Draft info...
Scrapping 2020 Draft info...
Scrapping 2019 Draft info...
Scrapping 2018 Draft info...
Scrapping 2017 Draft info...
Scrapping 2016 Draft info...
Scrapping 2015 Draft info...
Scrapping 2014 Draft info...
Scrapping 2013 Draft info...
Scrapping 2012 Draft info...
Scrapping 2011 Draft info...
Scrapping 2010 Draft info...
Scrapping 2009 Draft info...
Scrapping 2008 Draft info...
Scrapping 2007 Draft info...
Scrapping 2006 Draft info...
Scrapping 2005 Draft info...
Scrapping 2004 Draft info...
Scrapping 2003 Draft info...
Scrapping 2002 Draft info...
Scrapping 2001 Draft info...
Scrapping 2000 Draft info...
Scrapping 1999 Draft info...
Scrapping 1998 Draft info...
Scrapping 1997 Draft info...
Scrapping 1996 Draft info...
Scrapping 1995 Draft info...
Scrapping 1994 Draft info...
Scrapping 1993 Draft info...
Scrapping 1992 Draft info...
Scrapping 1991 Draft info...
Scrapping 1990 Draft info...
Scrapping 1989 Draft info...
Scrapping 1988 Draft info...
Scrapping 1987

In [74]:
print(len(draft))
draft.to_excel('prova_draft.xlsx') #Ho guardem a excel perquè cal fer algunes comprovacions pel data_cleaning

8150


### Observamos que hay muchos mas jugadores drafteados que del df original (jugadores con estadisticas). 

Razones: drafteado y no jugado, no drafteados (free agent), muy antiguos...

Aún así, hacemos el merge con el original

In [9]:
merge_df = original.merge(draft, on = 'player_url',how='left',suffixes=('','_draft'))

In [10]:
print(len(original))
print(len(merge_df))

4974
5046


In [11]:
merge_df_2 = merge_df.drop_duplicates(subset=['player_url'],keep='first',ignore_index=True) #Corretgim els sobrants

In [12]:
print(len(original))
print(len(merge_df_2))

4974
4974


In [13]:
# Hacer queries de ejemplos para ver que todo funciona mas o menos
merge_df_2[merge_df_2['draft_year'].isna()]

,player_url,player_name,active_from,active_to,position,college,height,weight,birth_date,draft_year,draft_round,draft_pick_number,draft_team,player_name_draft,college_draft
8,/players/a/abramjo01.html,John Abramovic,1947,1948,F,Salem University,6-3,195,"February 9, 1919",NaN,NaN,NaN,NaN,NaN,NaN
14,/players/a/actonbu01.html,Bud Acton,1968,1968,F,Hillsdale College,6-6,210,"January 11, 1942",NaN,NaN,NaN,NaN,NaN,NaN
20,/players/a/adamsja01.html,Jaylen Adams,2019,2021,G,St. Bonaventure,6-0,225,"May 4, 1996",NaN,NaN,NaN,NaN,NaN,NaN
26,/players/a/adelde01.html,Deng Adel,2019,2019,F,Louisville,6-7,200,"February 1, 1997",NaN,NaN,NaN,NaN,NaN,NaN
28,/players/a/adrieje01.html,Jeff Adrien,2011,2015,F,UConn,6-7,245,"February 10, 1986",NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4953,/players/y/yurtsom01.html,Omer Yurtseven,2022,2022,C,"NC State, Georgetown",7-0,264,"June 19, 1998",NaN,NaN,NaN,NaN,NaN,NaN
4954,/players/z/zasloma01.html,Max Zaslofsky,1947,1956,G-F,St. John's,6-2,170,"December 7, 1925",NaN,NaN,NaN,NaN,NaN,NaN
4959,/players/z/zelleha01.html,Harry Zeller,1947,1947,C-F,Washington & Jefferson College,6-4,210,"July 10, 1919",NaN,NaN,NaN,NaN,NaN,NaN
4960,/players/z/zellelu01.html,Luke Zeller,2013,2013,C,Notre Dame,6-11,245,"April 7, 1987",NaN,NaN,NaN,NaN,NaN,NaN


## Varias pruebas

In [17]:
url = '/players/b/bryanko01.html'
page_request = requests.get('https://www.basketball-reference.com' + url)
soup = BeautifulSoup(page_request.text,"lxml")

In [47]:
a = soup.find('ul',{'id':'bling'})
b = a.find('li',string = re.compile('All Star')).text

In [48]:
b[:b.find('x')]

'18'

In [27]:
# Per games stats (regular + playoff)

In [49]:
def player_info_extras(player_url):
    
    Hall_Of_Fame = 'No'
    n_NBA_Champ = 0
    n_All_Star = 0
    
    page_request = requests.get('https://www.basketball-reference.com' + str(player_url))
    soup = BeautifulSoup(page_request.text,"lxml")
    extras = soup.find('ul',{'id':'bling'})
    
    if extras:
        if extras.find('li',string = re.compile('Hall of Fame')) != None:  
            Hall_Of_Fame = 'Yes'
        if extras.find('li',string = re.compile('NBA Champ')) != None:
            aux = extras.find('li',string = re.compile('NBA Champ')).text
            n_NBA_Champ = aux[:aux.find('x')]
        if extras.find('li',string = re.compile('All Star')) != None:  
            aux = extras.find('li',string = re.compile('All Star')).text
            n_All_Star = aux[:aux.find('x')]
        
    player_info = {'player_url' : player_url,
                   'Hall_Of_Fame' : Hall_Of_Fame,
                   'n_NBA_Champ' : n_NBA_Champ,
                   'n_All_Star' : n_All_Star}
    
    return player_info

In [50]:
player_info_extras('/players/b/bryanko01.html')

{'player_url': '/players/b/bryanko01.html',
 'Hall_Of_Fame': 'Yes',
 'n_NBA_Champ': '5',
 'n_All_Star': '18'}

In [53]:
prova = original['player_url'][:30]
df_extras= []
start_timer_1 = time.perf_counter()
diff_time = 0
for i in prova:
    start_timer = time.perf_counter()
    df_extras.append(player_info_extras(i))
    end_timer = time.perf_counter()
    diff_time = diff_time + (end_timer-start_timer)
    #print(diff_time)
df_extras = pd.DataFrame(df_extras)
#print(str((end_timer-start_timer_1)))
print('Tiempo de media scrapeado ' + str(diff_time/len(prova)))

Tiempo de media scrapeado 0.48964178999998087


In [54]:
df_extras

,player_url,Hall_Of_Fame,n_NBA_Champ,n_All_Star
0,/players/a/abdelal01.html,No,0,0
1,/players/a/abdulza01.html,No,0,0
2,/players/a/abdulka01.html,Yes,6,19
3,/players/a/abdulma02.html,No,0,0
4,/players/a/abdulta01.html,No,0,0
5,/players/a/abdursh01.html,No,0,1
6,/players/a/abernto01.html,No,0,0
7,/players/a/ablefo01.html,No,0,0
8,/players/a/abramjo01.html,No,0,0
9,/players/a/abrinal01.html,No,0,0


In [43]:
#pruebas con tablas

In [11]:
player_info_advanced('/players/b/bryanko01.html','advanced','_NBA_Reg')

{'player_url': '/players/b/bryanko01.html',
 'player_eff_rating_NBA_Reg': '22.9',
 'true_shooting_eff_NBA_Reg': '.550',
 '_3pts_attempts_rate_NBA_Reg': '.212',
 'FT_attempts_rate_NBA_Reg': '.382',
 'off_rebounds_percent_NBA_Reg': '3.5',
 'def_rebounds_percent_NBA_Reg': '12.7',
 'total_rebounds_percent_NBA_Reg': '8.1',
 'assists_percent_NBA_Reg': '24.2',
 'steals_percent_NBA_Reg': '2.1',
 'blocks_percent_NBA_Reg': '1.0',
 'turnover_percent_NBA_Reg': '11.6',
 'usage_percent_NBA_Reg': '31.8',
 'off_win_share_NBA_Reg': '122.1',
 'def_win_share_NBA_Reg': '50.7',
 'win_share_NBA_Reg': '172.7',
 'win_share_48min_NBA_Reg': '.170',
 'off_box_pm_NBA_Reg': '4.6',
 'def_box_pm_NBA_Reg': '-0.1',
 'value_over_replacement_NBA_Reg': '4.6'}

In [12]:
prova = original['player_url'][:10]
df_reg = []
df_playoff = []
start_timer_1 = time.perf_counter()
diff_time = 0
for i in prova:
    start_timer = time.perf_counter()
    df_reg.append(player_info_advanced(i,'advanced','_NBA_Reg'))
    df_playoff.append(player_info_advanced(i,'playoffs_advanced','_NBA_Playoff'))
    end_timer = time.perf_counter()
    diff_time = diff_time + (end_timer-start_timer)
    #print(diff_time)
df_reg = pd.DataFrame(df_reg)
df_playoff = pd.DataFrame(df_playoff)
#print(str((end_timer-start_timer_1)))
print('Tiempo de media scrapeado ' + str(diff_time/len(prova)))

Tiempo de media scrapeado 2.0629795199999537


In [13]:
df_reg

,player_url,player_eff_rating_NBA_Reg,true_shooting_eff_NBA_Reg,_3pts_attempts_rate_NBA_Reg,FT_attempts_rate_NBA_Reg,off_rebounds_percent_NBA_Reg,def_rebounds_percent_NBA_Reg,total_rebounds_percent_NBA_Reg,assists_percent_NBA_Reg,steals_percent_NBA_Reg,blocks_percent_NBA_Reg,turnover_percent_NBA_Reg,usage_percent_NBA_Reg,off_win_share_NBA_Reg,def_win_share_NBA_Reg,win_share_NBA_Reg,win_share_48min_NBA_Reg,off_box_pm_NBA_Reg,def_box_pm_NBA_Reg,value_over_replacement_NBA_Reg
0,/players/a/abdelal01.html,13.0,.532,.002,.260,10.0,19.6,14.8,4.1,1.1,1.4,15.2,21.6,0.7,4.1,4.8,.072,-2.9,-0.9,-3.8
1,/players/a/abdulza01.html,15.1,.479,None,.338,12.1,26.9,18.3,7.3,1.3,2.5,16.6,21.1,5.9,11.6,17.5,.076,0.6,-0.2,0.4
2,/players/a/abdulka01.html,24.6,.592,.001,.329,7.7,21.7,15.7,14.6,1.2,3.8,13.4,24.3,178.9,94.5,273.4,.228,4.1,1.6,5.7
3,/players/a/abdulma02.html,15.4,.506,.169,.146,1.5,6.2,3.8,23.0,1.6,0.2,10.2,25.4,16.7,8.4,25.2,.077,0.7,-1.5,-0.8
4,/players/a/abdulta01.html,11.4,.467,.044,.306,6.4,11.2,8.8,8.8,1.9,1.2,13.6,20.4,-0.6,4.1,3.5,.035,-2.6,-0.4,-3.0
5,/players/a/abdursh01.html,19.0,.549,.045,.429,7.5,17.5,12.5,13.6,1.5,1.6,13.5,25.0,50.1,21.1,71.2,.118,1.7,-0.7,1.0
6,/players/a/abernto01.html,12.9,.534,.001,.301,7.3,12.3,9.8,9.0,1.6,0.6,9.3,13.7,8.4,5.0,13.4,.118,0.0,0.1,0.2
7,/players/a/ablefo01.html,-41.0,.000,None,.000,,,,,None,None,None,None,0.0,0.0,0.0,-2.062,None,None,None
8,/players/a/abramjo01.html,,.285,None,.216,,,,,None,None,None,None,-1.4,-0.5,-1.9,,None,None,None
9,/players/a/abrinal01.html,8.8,.552,.754,.137,1.9,8.0,4.9,4.4,1.6,0.7,7.9,13.8,2.5,2.5,5.0,.086,-2.1,0.1,-1.9


In [14]:
df_playoff

,player_url,player_eff_rating_NBA_Playoff,true_shooting_eff_NBA_Playoff,_3pts_attempts_rate_NBA_Playoff,FT_attempts_rate_NBA_Playoff,off_rebounds_percent_NBA_Playoff,def_rebounds_percent_NBA_Playoff,total_rebounds_percent_NBA_Playoff,assists_percent_NBA_Playoff,steals_percent_NBA_Playoff,blocks_percent_NBA_Playoff,turnover_percent_NBA_Playoff,usage_percent_NBA_Playoff,off_win_share_NBA_Playoff,def_win_share_NBA_Playoff,win_share_NBA_Playoff,win_share_48min_NBA_Playoff,off_box_pm_NBA_Playoff,def_box_pm_NBA_Playoff,value_over_replacement_NBA_Playoff
0,/players/a/abdelal01.html,4.8,.455,.000,.100,3.5,20.1,11.8,4.4,0.0,0.6,20.8,22.1,-0.3,0.1,-0.2,-0.098,-6.7,-2.3,-9.1
1,/players/a/abdulza01.html,17.1,.565,None,.371,9.8,24.9,17.2,5.8,0.4,3.4,,,0.5,0.1,0.5,.124,1.7,-0.4,1.3
2,/players/a/abdulka01.html,23.0,.571,.001,.321,8.3,18.7,14.5,13.2,1.3,3.7,12.3,25.6,21.8,13.7,35.6,.193,4.2,1.1,5.3
3,/players/a/abdulma02.html,11.5,.471,.262,.241,1.4,5.0,3.2,17.0,0.9,0.2,9.6,25.3,0.2,0.0,0.2,.026,-0.8,-2.3,-3.1
4,/players/a/abdulta01.html,17.0,.462,.048,.381,10.6,15.6,13.0,10.3,1.2,1.5,5.2,19.8,0.3,0.1,0.4,.112,1.8,-0.3,1.5
5,/players/a/abdursh01.html,11.3,.554,.070,.349,7.1,22.9,14.2,10.2,0.8,0.0,20.8,22.8,0.0,0.0,-0.1,-0.030,-3.5,-1.9,-5.4
6,/players/a/abernto01.html,12.4,.509,,.537,6.4,12.7,9.6,13.5,1.4,0.5,0.0,16.1,0.2,0.2,0.4,.078,0.3,-0.7,-0.4
7,/players/a/ablefo01.html,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
8,/players/a/abramjo01.html,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None,None
9,/players/a/abrinal01.html,8.4,.526,.698,.140,2.8,11.4,7.0,4.5,1.3,1.0,2.1,10.6,0.2,0.1,0.3,.076,-2.0,0.3,-1.7


In [55]:
def prova(url):
    page_request = requests.get('https://www.basketball-reference.com' + url)
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find("table",{'id':'per_game'})
    table_foot = table.find('tfoot')
    career_stats = table_foot.find('tr').findAll('td',{'class':'right'})
    stats_vector = []
    stats_vector.append(url)
    for i in range(0,len(career_stats)):
        try:
            stats_vector.append(career_stats[i]['data-stat']) #ho converteixes tot a float y després apliques lógica als que son int
        except:
            stats_vector.append('')
    return stats_vector

In [57]:
empty_df = pd.DataFrame(columns=['url','1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25'])

number = 0
for url in original['player_url']:
    empty_df = empty_df.append(dict(zip(empty_df.columns, prova(url))), ignore_index=True)
    number = number +1
    if number % 100 == 0:
        print(number)


100
200
300
400
500


KeyboardInterrupt: 

In [58]:
empty_df

,url,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
0,/players/a/abdelal01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
1,/players/a/abdulza01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,ft_per_g,fta_per_g,ft_pct,...,tov_per_g,pf_per_g,pts_per_g,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,/players/a/abdulka01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
3,/players/a/abdulma02.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
4,/players/a/abdulta01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
574,/players/b/brownst02.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
575,/players/b/brownti01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
576,/players/b/brownto01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g
577,/players/b/browntr01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,stl_per_g,blk_per_g,tov_per_g,pf_per_g,pts_per_g


In [87]:
empty_df[(empty_df['24'].isna()) & (empty_df['19'].isna()==False)]
#14 -> (88)15% (not rebounds)
#18 -> (15)2% (not 3's)
#19 -> (15)2% (not 3's)
#24 -> (23)4% (not steals or blocks)

,url,1,2,3,4,5,6,7,8,9,...,16,17,18,19,20,21,22,23,24,25
34,/players/a/aitchma01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
36,/players/a/akinhe01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
58,/players/a/allenbi01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
87,/players/a/anderan01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
89,/players/a/andercl01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
90,/players/a/anderda01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
162,/players/a/austijo01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
208,/players/b/bankswa01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
273,/players/b/bayneho01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN
280,/players/b/beardal01.html,g,gs,mp_per_g,fg_per_g,fga_per_g,fg_pct,fg3_per_g,fg3a_per_g,fg3_pct,...,ft_pct,orb_per_g,drb_per_g,trb_per_g,ast_per_g,tov_per_g,pf_per_g,pts_per_g,NaN,NaN


In [93]:
a = [None,1]
a
pd.DataFrame(a,columns=['url'])

,url
0,NaN
1,1.0


In [90]:
a +'_er' = 2

SyntaxError: cannot assign to operator (<ipython-input-90-5b73c5132f11>, line 1)

In [40]:
for i in range(1,26):
    print("'"+ str(i) +"'",end=",")

'1','2','3','4','5','6','7','8','9','10','11','12','13','14','15','16','17','18','19','20','21','22','23','24','25',

In [8]:
def player_info(player_url):
    #define all quantites
    fgpct = None
    games = None
    ppg = None
    ft = None
    fgpg = None
    fgapg = None
    ftpg = None
    ftapg = None
    _3ptpg = None
    _3ptapg = None
    _3ptpct = None
    efgpct = None
    NCAA_fgpct = None
    NCAA_games = None
    NCAA_ppg = None
    NCAA_ft = None
    NCAA_fgpg = None
    NCAA_fgapg = None
    NCAA_ftpg = None
    NCAA_ftapg = None
    NCAA__3ptpg = None
    NCAA__3ptapg = None
    NCAA__3ptpct = None
    NCAA_efgpct = None
    page_request = requests.get('http://www.basketball-reference.com' + str(url))
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find('table') #the first table is luckily the per game stats
    if table:
            table_foot = table.find('tfoot')
            for row in table_foot.findAll('tr'):
                cells  = row.findAll('td')
                playerData = str(cells) #the indexes are not uniform across the database
                #games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1) # don't need
                fgpct = re.search(r'data-stat="fg_pct">(.*?)</td>', playerData).group(1)
                games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1)
                ppg = re.search(r'data-stat="pts_per_g">(.*?)</td>', playerData).group(1)
                ft = re.search(r'data-stat="ft_pct">(.*?)</td>', playerData).group(1)
                fgpg = re.search(r'data-stat="fg_per_g">(.*?)</td>', playerData).group(1)
                fgapg = re.search(r'data-stat="fga_per_g">(.*?)</td>', playerData).group(1)
                ftpg = re.search(r'data-stat="ft_per_g">(.*?)</td>', playerData).group(1)
                ftapg = re.search(r'data-stat="fta_per_g">(.*?)</td>', playerData).group(1)
                if re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData) != None:
                    _3ptpg = re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData).group(1)
                else:
                    _3ptpg = None
                if re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData) != None:
                    _3ptapg = re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData).group(1)
                else:
                    _3ptapg = None
                if re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData) != None:
                    _3ptpct = re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData).group(1)	
                else:
                    _3ptpct = None
                if re.search(r'data-stat="efg_pct">(.*?)</td>', playerData) != None:
                    efgpct = re.search(r'data-stat="efg_pct">(.*?)</td>', playerData).group(1)
                else:
                    efgpct = None
                break  #bad but I want the structure to remain the same in case I want more data outside overall stats

In [5]:
page_request = requests.get('https://www.basketball-reference.com/draft/NBA_2021.html')
soup = BeautifulSoup(page_request.text,"lxml")

In [6]:
table = soup.find("table")

In [7]:
table_body = table.find('tbody')
#for row in table_body.findAll('tr'):
#    row.find('a')

In [27]:
round = 1
for row in table_body.findAll('tr'):
    try:
        cells = row.findAll('td')
        print(cells[0].text + " " + cells[1].text + " " + cells[2].text + " " + cells[3].text )
        #print(cells[2].a['href'])
    except:
        round = round + 1
        pass

1 DET Cade Cunningham Oklahoma State
2 HOU Jalen Green 
3 CLE Evan Mobley USC
4 TOR Scottie Barnes Florida State
5 ORL Jalen Suggs Gonzaga
6 OKC Josh Giddey 
7 GSW Jonathan Kuminga 
8 ORL Franz Wagner Michigan
9 SAC Davion Mitchell Baylor
10 NOP Ziaire Williams Stanford
11 CHO James Bouknight UConn
12 SAS Joshua Primo Alabama
13 IND Chris Duarte Oregon
14 GSW Moses Moody Arkansas
15 WAS Corey Kispert Gonzaga
16 OKC Alperen Şengün 
17 MEM Trey Murphy III Virginia
18 OKC Tre Mann Florida
19 NYK Kai Jones Texas
20 ATL Jalen Johnson Duke
21 NYK Keon Johnson Tennessee
22 LAL Isaiah Jackson Kentucky
23 HOU Usman Garuba 
24 HOU Josh Christopher Arizona State
25 LAC Quentin Grimes Houston
26 DEN Bones Hyland VCU
27 BRK Cameron Thomas LSU
28 PHI Jaden Springer Tennessee
29 PHO Day'Ron Sharpe UNC
30 UTA Santi Aldama Loyola (MD)
31 MIL Isaiah Todd 
32 NYK Jeremiah Robinson-Earl Villanova
33 ORL Jason Preston Ohio University
34 OKC Rokas Jokubaitis 
35 NOP Herb Jones Alabama
36 OKC Miles McBride W

In [20]:
round

3

In [19]:
soup.find_all(href = re.compile('/teams/'))

[]

In [45]:
a = soup.find_all("th",{"data-stat" : "draft"})[1:]
draft_urls = []
for i in a:
    draft_urls.append(i.a['href'])

In [46]:
draft_urls

['/draft/NBA_2021.html',
 '/draft/NBA_2020.html',
 '/draft/NBA_2019.html',
 '/draft/NBA_2018.html',
 '/draft/NBA_2017.html',
 '/draft/NBA_2016.html',
 '/draft/NBA_2015.html',
 '/draft/NBA_2014.html',
 '/draft/NBA_2013.html',
 '/draft/NBA_2012.html',
 '/draft/NBA_2011.html',
 '/draft/NBA_2010.html',
 '/draft/NBA_2009.html',
 '/draft/NBA_2008.html',
 '/draft/NBA_2007.html',
 '/draft/NBA_2006.html',
 '/draft/NBA_2005.html',
 '/draft/NBA_2004.html',
 '/draft/NBA_2003.html',
 '/draft/NBA_2002.html',
 '/draft/NBA_2001.html',
 '/draft/NBA_2000.html',
 '/draft/NBA_1999.html',
 '/draft/NBA_1998.html',
 '/draft/NBA_1997.html',
 '/draft/NBA_1996.html',
 '/draft/NBA_1995.html',
 '/draft/NBA_1994.html',
 '/draft/NBA_1993.html',
 '/draft/NBA_1992.html',
 '/draft/NBA_1991.html',
 '/draft/NBA_1990.html',
 '/draft/NBA_1989.html',
 '/draft/NBA_1988.html',
 '/draft/NBA_1987.html',
 '/draft/NBA_1986.html',
 '/draft/NBA_1985.html',
 '/draft/NBA_1984.html',
 '/draft/NBA_1983.html',
 '/draft/NBA_1982.html',


In [138]:
example = player_basic_info()

Scrapping basic info of players with letter a...
Scrapping basic info of players with letter b...
Scrapping basic info of players with letter c...
Scrapping basic info of players with letter d...
Scrapping basic info of players with letter e...
Scrapping basic info of players with letter f...
Scrapping basic info of players with letter g...
Scrapping basic info of players with letter h...
Scrapping basic info of players with letter i...
Scrapping basic info of players with letter j...
Scrapping basic info of players with letter k...
Scrapping basic info of players with letter l...
Scrapping basic info of players with letter m...
Scrapping basic info of players with letter n...
Scrapping basic info of players with letter o...
Scrapping basic info of players with letter p...
Scrapping basic info of players with letter q...
Scrapping basic info of players with letter r...
Scrapping basic info of players with letter s...
Scrapping basic info of players with letter t...
Scrapping basic info

In [139]:
copia_example = example.copy()

In [140]:
zzz = copia_example[:10]

In [141]:
zzz

,url,name,active_from,active_to,position,college,height,weight,birth_date
0,/players/a/abdelal01.html,Alaa Abdelnaby,1991,1995,F-C,Duke,6-10,240,"June 24, 1968"
1,/players/a/abdulza01.html,Zaid Abdul-Aziz,1969,1978,C-F,Iowa State,6-9,235,"April 7, 1946"
2,/players/a/abdulka01.html,Kareem Abdul-Jabbar,1970,1989,C,UCLA,7-2,225,"April 16, 1947"
3,/players/a/abdulma02.html,Mahmoud Abdul-Rauf,1991,2001,G,LSU,6-1,162,"March 9, 1969"
4,/players/a/abdulta01.html,Tariq Abdul-Wahad,1998,2003,F,"Michigan, San Jose State",6-6,223,"November 3, 1974"
5,/players/a/abdursh01.html,Shareef Abdur-Rahim,1997,2008,F,California,6-9,225,"December 11, 1976"
6,/players/a/abernto01.html,Tom Abernethy,1977,1981,F,Indiana,6-7,220,"May 6, 1954"
7,/players/a/ablefo01.html,Forest Able,1957,1957,G,Western Kentucky,6-3,180,"July 27, 1932"
8,/players/a/abramjo01.html,John Abramovic,1947,1948,F,Salem University,6-3,195,"February 9, 1919"
9,/players/a/abrinal01.html,Álex Abrines,2017,2019,G-F,,6-6,200,"August 1, 1993"


In [143]:
get_player_draft_info('/players/a/abdelal01.html')

'Portland Trail Blazers'

In [144]:
zzz['draft_team'] = zzz.apply(lambda x: get_player_draft_info(x['url']) ,axis=1)

<ipython-input-144-8b70c12c897f>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  zzz['draft_team'] = zzz.apply(lambda x: get_player_draft_info(x['url']) ,axis=1)


In [145]:
zzz

,url,name,active_from,active_to,position,college,height,weight,birth_date,draft_team
0,/players/a/abdelal01.html,Alaa Abdelnaby,1991,1995,F-C,Duke,6-10,240,"June 24, 1968",Portland Trail Blazers
1,/players/a/abdulza01.html,Zaid Abdul-Aziz,1969,1978,C-F,Iowa State,6-9,235,"April 7, 1946",Cincinnati Royals
2,/players/a/abdulka01.html,Kareem Abdul-Jabbar,1970,1989,C,UCLA,7-2,225,"April 16, 1947",Milwaukee Bucks
3,/players/a/abdulma02.html,Mahmoud Abdul-Rauf,1991,2001,G,LSU,6-1,162,"March 9, 1969",Denver Nuggets
4,/players/a/abdulta01.html,Tariq Abdul-Wahad,1998,2003,F,"Michigan, San Jose State",6-6,223,"November 3, 1974",Sacramento Kings
5,/players/a/abdursh01.html,Shareef Abdur-Rahim,1997,2008,F,California,6-9,225,"December 11, 1976",Vancouver Grizzlies
6,/players/a/abernto01.html,Tom Abernethy,1977,1981,F,Indiana,6-7,220,"May 6, 1954",Los Angeles Lakers
7,/players/a/ablefo01.html,Forest Able,1957,1957,G,Western Kentucky,6-3,180,"July 27, 1932",Syracuse Nationals
8,/players/a/abramjo01.html,John Abramovic,1947,1948,F,Salem University,6-3,195,"February 9, 1919",
9,/players/a/abrinal01.html,Álex Abrines,2017,2019,G-F,,6-6,200,"August 1, 1993",Oklahoma City Thunder


In [146]:
url = '/players/r/rubiori01.html' 

In [147]:
page_request = requests.get('http://www.basketball-reference.com' + str(url))
soup = BeautifulSoup(page_request.text,"lxml")

In [148]:
soup

<!DOCTYPE html>
<html class="no-js" data-root="/home/bbr/build" data-version="klecko-" itemscope="" itemtype="https://schema.org/WebSite" lang="en">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<meta content="width=device-width, initial-scale=1.0, maximum-scale=2.0" name="viewport"/>
<link href="https://d2p3bygnnzw9w3.cloudfront.net/req/202112021" rel="dns-prefetch"/>
<!-- Quantcast Choice. Consent Manager Tag v2.0 (for TCF 2.0) -->
<script async="true" type="text/javascript">
    (function() {
	var host = window.location.hostname;
	var element = document.createElement('script');
	var firstScript = document.getElementsByTagName('script')[0];
	var url = 'https://quantcast.mgr.consensu.org'
	    .concat('/choice/', 'XwNYEpNeFfhfr', '/', host, '/choice.js')
	var uspTries = 0;
	var uspTriesLimit = 3;
	element.async = true;
	element.type = 'text/javascript';
	element.src = url;
	
	firstScript.parentNode.insertBefore(element, firstScript);
	
	function

In [164]:
a = soup.find_all("p")
 = ["Draft" in i.text for i in a]

[False,
 False,
 False,
 False,
 False,
 True,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False,
 False]

In [118]:
soup.find("strong",text="\n  Draft:\n  ")

'\n  Draft:\n  '

In [124]:
soup.find(href = re.compile("/draft.html")).get_text()

'Minnesota Timberwolves'

In [122]:
soup2 = soup.find_all("strong")
BeautifulSoup(soup2.text,"lxml")

AttributeError: ResultSet object has no attribute 'text'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [ ]:
# Aquí cal fer una selecció de quines taules volem treure (ojo perq alomillor val la pena fer la distinció de Play-off y regular). A més també caldria treure número d'equips i quins equips (després es pot fer un get dummies dels equips amb més títols). Draft a la descripció. NBA champ al cartellet.

In [ ]:
def player_info(url):
    #define all quantites
    fgpct = None
    games = None
    ppg = None
    ft = None
    fgpg = None
    fgapg = None
    ftpg = None
    ftapg = None
    _3ptpg = None
    _3ptapg = None
    _3ptpct = None
    efgpct = None
    NCAA_fgpct = None
    NCAA_games = None
    NCAA_ppg = None
    NCAA_ft = None
    NCAA_fgpg = None
    NCAA_fgapg = None
    NCAA_ftpg = None
    NCAA_ftapg = None
    NCAA__3ptpg = None
    NCAA__3ptapg = None
    NCAA__3ptpct = None
    NCAA_efgpct = None
    #print('url = ' + str('http://www.basketball-reference.com' + str(url)))
    page_request = requests.get('http://www.basketball-reference.com' + str(url))
    soup = BeautifulSoup(page_request.text,"lxml")
    table = soup.find('table') #the first table is luckily the per game stats
    if table:
            table_foot = table.find('tfoot')
            for row in table_foot.findAll('tr'):
                cells  = row.findAll('td')
                playerData = str(cells) #the indexes are not uniform across the database
                #games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1) # don't need
                fgpct = re.search(r'data-stat="fg_pct">(.*?)</td>', playerData).group(1)
                games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1)
                ppg = re.search(r'data-stat="pts_per_g">(.*?)</td>', playerData).group(1)
                ft = re.search(r'data-stat="ft_pct">(.*?)</td>', playerData).group(1)
                fgpg = re.search(r'data-stat="fg_per_g">(.*?)</td>', playerData).group(1)
                fgapg = re.search(r'data-stat="fga_per_g">(.*?)</td>', playerData).group(1)
                ftpg = re.search(r'data-stat="ft_per_g">(.*?)</td>', playerData).group(1)
                ftapg = re.search(r'data-stat="fta_per_g">(.*?)</td>', playerData).group(1)
                if re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData) != None:
                    _3ptpg = re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData).group(1)
                else:
                    _3ptpg = None
                if re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData) != None:
                    _3ptapg = re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData).group(1)
                else:
                    _3ptapg = None
                if re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData) != None:
                    _3ptpct = re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData).group(1)	
                else:
                    _3ptpct = None
                if re.search(r'data-stat="efg_pct">(.*?)</td>', playerData) != None:
                    efgpct = re.search(r'data-stat="efg_pct">(.*?)</td>', playerData).group(1)
                else:
                    efgpct = None
                break  #bad but I want the structure to remain the same in case I want more data outside overall stats

    college_url = get_player_college_url(url)
    if(college_url != None):
                page_request_cbb = requests.get(college_url)
                soupy = BeautifulSoup(page_request_cbb.text,'lxml')
                table_cbb = soupy.find('table')
                if table_cbb:
                	table_foot = table_cbb.find('tfoot')
                	for row in table_foot.findAll('tr'):
                                cells  = row.findAll('td')
                                playerData = str(cells) #the indexes are not uniform across the database
                                NCAA_fgpct = re.search(r'data-stat="fg_pct">(.*?)</td>', playerData).group(1)
                                NCAA_games = re.search(r'data-stat="g">(.*?)</td>', playerData).group(1)
                                NCAA_ppg = re.search(r'data-stat="pts_per_g">(.*?)</td>', playerData).group(1)
                                NCAA_ft = re.search(r'data-stat="ft_pct">(.*?)</td>', playerData).group(1)
                                NCAA_fgpg = re.search(r'data-stat="fg_per_g">(.*?)</td>', playerData).group(1)
                                NCAA_fgapg = re.search(r'data-stat="fga_per_g">(.*?)</td>', playerData).group(1)
                                NCAA_ftpg = re.search(r'data-stat="ft_per_g">(.*?)</td>', playerData).group(1)
                                NCAA_ftapg = re.search(r'data-stat="fta_per_g">(.*?)</td>', playerData).group(1)
                                if re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData) != None:
                                	NCAA__3ptpg = re.search(r'data-stat="fg3_per_g">(.*?)</td>', playerData).group(1)
                                else:
                                	NCAA__3ptpg = None
                                if re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData) != None:
                                	NCAA__3ptapg = re.search(r'data-stat="fg3a_per_g">(.*?)</td>', playerData).group(1)
                                else:
                                	NCAA__3ptapg = None
                                if re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData) != None:
                                	NCAA__3ptpct = re.search(r'data-stat="fg3_pct">(.*?)</td>', playerData).group(1)	
                                else:
                                	NCAA__3ptpct = None
                                if re.search(r'data-stat="efg_pct">(.*?)</td>', playerData) != None:
                                	NCAA_efgpct = re.search(r'data-stat="efg_pct">(.*?)</td>', playerData).group(1)
                                else:
                                	NCAA_efgpct = None
                                break

	player_entry = {'NBA_fg%':fgpct ,
	                'NBA_g_played': games,
	                'NBA_ppg': ppg,
	                'NBA_ft%': ft,
	                'NBA_fg_per_game': fgpg,
	                'NBA_fga_per_game': fgapg,
	                'NBA_ft_per_g': ftpg,
	                'NBA_fta_p_g': ftapg,
	                'NBA__3ptpg': _3ptpg,
	                'NBA__3ptapg': _3ptapg,
	                'NBA__3ptpct': _3ptpct,
	                'NBA_efgpct': efgpct,
	                'NCAA_fgpct': NCAA_fgpct,
	                'NCAA_games': NCAA_games,
	                'NCAA_ppg' : NCAA_ppg,
	                'NCAA_ft': NCAA_ft,
	                'NCAA_fgpg': NCAA_fgpg,
	                'NCAA_fgapg': NCAA_fgapg,
	                'NCAA_ftpg': NCAA_ftpg,
	                'NCAA_ftapg': NCAA_ftapg,
	                'NCAA__3ptpg': NCAA__3ptpg,
	                'NCAA__3ptapg': NCAA__3ptapg,
	                'NCAA__3ptpct': NCAA__3ptpct,
	                'NCAA_efgpct': NCAA_efgpct
	                }
    
	#print('player ' + url + 'complete')
	#print(player_entry)
	return player_entry

In [ ]:
def get_player_college_url(NBA_url):
	page_request = requests.get('http://www.basketball-reference.com' + str(url))
	soup = BeautifulSoup(page_request.text,"lxml")
	links = str(soup.findAll('li')) #regex time
	college_url = re.search(r'<a href="(.*?)">College Basketball at Sports-Reference.com</a>', links)
	if(college_url != None):
		return str(college_url.group(1))
	else:
		return None
	

######################################################################################
#MAIN 
players_general_info = player_basic_info() # call function that scrapes general info
print('General info/player url loaded...')
players_details_info_list = []
df = pd.DataFrame()
bar = progressbar.ProgressBar(max_value=len(players_general_info))
for i,url in enumerate(players_general_info.url):
	player = player_info(url)
	df = df.append(player, ignore_index = True)
	print(df)
	bar.update(i)
	time.sleep(0.1)
print('Done!') #takes an unholy amount of time
df = pd.concat([players_general_info, df], axis =1, join_axes=[df.index])
df.to_csv('players.csv', encoding='utf-8')


######################################################################################


 #print(players_details_info_list[i])
# for i,url in enumerate(players_general_info.url):
# 	try:
# 	    players_details_info_list.append(player_detail_info(url))
# 	    print(players_details_info_list[i])
# 	except:
# 		print('cannot load: %s; location %d' %(url,i)) 
ABOUT THIS FILE
Last Updated
3 years ago
Owner
Ben Pierce
Created
3 years ago
Size
8.2 KB


In [ ]:
# Proves fallides!!!!

In [137]:
# Prova, va massa lent....
def get_player_draft_info(url):
    page_request = requests.get('http://www.basketball-reference.com' + str(url))
    soup = BeautifulSoup(page_request.text,"lxml")
    try:
        draft_team = soup.find(href = re.compile("/draft.html")).get_text()
    except:
        draft_team = ''
    return draft_team